In [1]:
import pandas as pd
import numpy as np
import lightgbm
import matplotlib.pyplot as plt
from pathlib import Path
import random
from IPython.display import display
from sklearn.model_selection import train_test_split, GroupKFold, KFold
from sklearn.metrics import mean_absolute_error
import optuna
from tensorflow import keras
import tensorflow as tf
from sklearn.preprocessing import normalize
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

2021-10-03 08:07:41.452277: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-03 08:07:41.452386: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from tensorflow import keras
from tensorflow.keras import layers

In [5]:
import wandb
from wandb.keras import WandbCallback

In [6]:
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("wandb_api")
    wandb.login(key=secret_value_0)
    anony=None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. 
Get your W&B access token from here: https://wandb.ai/authorize


In [7]:
from datetime import datetime
import gc

In [8]:
tdate = datetime.now().strftime("%Y%m%d_%H%M%S")
tdate

'20211003_080746'

In [9]:
from tensorflow.keras.callbacks import Callback
import tensorflow.keras.backend as K
class WarmupExponentialDecay(Callback):
    def __init__(self,lr_base=0.0002,lr_min=0.0,decay=0,warmup_epochs=0):
        self.num_passed_batchs = 0   #一个计数器
        self.warmup_epochs=warmup_epochs  
        self.lr=lr_base #learning_rate_base
        self.lr_min=lr_min #最小的起始学习率,此代码尚未实现
        self.decay=decay  #指数衰减率
        self.steps_per_epoch=0 #也是一个计数器
        
    def on_batch_begin(self, batch, logs=None):
        # params是模型自动传递给Callback的一些参数
        if self.steps_per_epoch==0:
            #防止跑验证集的时候呗更改了
            if self.params['steps'] == None:
                self.steps_per_epoch = np.ceil(1. * self.params['samples'] / self.params['batch_size'])
            else:
                self.steps_per_epoch = self.params['steps']
        if self.num_passed_batchs < self.steps_per_epoch * self.warmup_epochs:
            K.set_value(self.model.optimizer.lr,
                        self.lr*(self.num_passed_batchs + 1) / self.steps_per_epoch / self.warmup_epochs)
        else:
            K.set_value(self.model.optimizer.lr,
                        self.lr*((1-self.decay)**(self.num_passed_batchs-self.steps_per_epoch*self.warmup_epochs)))
        self.num_passed_batchs += 1
        
    def on_epoch_begin(self,epoch,logs=None):
        #用来输出学习率的,可以删除
        print("learning_rate:",K.get_value(self.model.optimizer.lr))

In [10]:
test = np.load('/kaggle/input/vpp-train-test-processed/test_data.npy')

In [11]:
home = 'vpp-train-test-processed'
test_preds = []
EPOCH = 1000
BATCH_SIZE = 1024

tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

CONFIG ={  # and include hyperparameters and metadata
                     "learning_rate": 1e-3,
                     "epochs": EPOCH,
                     "batch_size": BATCH_SIZE,
                     "loss_function": "mae",
                     "architecture": "lstm",
                     "dataset": "vpp-dataset-kaggle",
                     "lstm_layers":[400,300,300,250,150,100,'50D'],
                 }
project = f'VentilatorPressurePrediction_kaggle_shadab_{tdate}'

for FOLD in range(2,4):
    print('---'*15, '>', f'Fold {FOLD}', '<', '---'*15)
    run = wandb.init(project=project, 
                     name='train_data', 
                     anonymous=anony,
                     config=CONFIG)
    config = wandb.config
    with tpu_strategy.scope():
        X_train, X_valid = np.load(f'/kaggle/input/{home}/X_train_{FOLD}.npy'),np.load(f'/kaggle/input/{home}/X_valid_{FOLD}.npy')
        y_train, y_valid = np.load(f'/kaggle/input/{home}/y_train_{FOLD}.npy'),np.load(f'/kaggle/input/{home}/y_valid_{FOLD}.npy')

    
        #scheduler = tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 400*((len(train)*0.8)/BATCH_SIZE), 1e-5)
        
        
        model = keras.models.Sequential([
            keras.layers.Input(shape=X_train.shape[-2:]),
            keras.layers.Bidirectional(keras.layers.LSTM(1024, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),
            keras.layers.Dense(128, activation='selu'),
            keras.layers.Dense(1),
        ])

    
        model.compile(optimizer="adam", loss='mae')
        
        cb1 = tf.keras.callbacks.ModelCheckpoint(
                    "/kaggle/working/best_model.h5", save_best_only=True, monitor="val_loss",mode="min")
        
        #cb2 = tf.keras.callbacks.LearningRateScheduler(scheduler)
        
        es = EarlyStopping(monitor="val_loss", patience=60, verbose=1, mode="min", restore_best_weights=True)
        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
        
        
        model.fit(X_train, y_train, 
              validation_data=(X_valid, y_valid), 
              epochs=EPOCH, 
              batch_size=BATCH_SIZE, 
              callbacks=[lr, WandbCallback()])#
        
        #model = tf.keras.models.load_model("/kaggle/working/best_model.h5")
        del X_train,X_valid,y_train,y_valid
        gc.collect()
        
        test_preds.append(model.predict(test).squeeze().reshape(-1, 1).squeeze())
        
wandb.finish()

2021-10-03 08:07:58.652672: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-03 08:07:58.655747: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-03 08:07:58.655778: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-03 08:07:58.655803: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cc1dc638de55): /proc/driver/nvidia/version does not exist
2021-10-03 08:07:58.658321: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

--------------------------------------------- > Fold 2 < ---------------------------------------------


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: wandb version 0.12.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-10-03 08:08:06.201397: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-03 08:08:06.201633: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



Epoch 1/1000
59/59 [==============================] - 64s 578ms/step - loss: 4.6225 - val_loss: 1.2942
Epoch 2/1000
59/59 [==============================] - 11s 179ms/step - loss: 1.2138 - val_loss: 0.7812
Epoch 3/1000
59/59 [==============================] - 11s 179ms/step - loss: 0.8040 - val_loss: 0.6707
Epoch 4/1000
59/59 [==============================] - 11s 179ms/step - loss: 0.6166 - val_loss: 0.6437
Epoch 5/1000
59/59 [==============================] - 11s 179ms/step - loss: 0.5672 - val_loss: 0.4923
Epoch 6/1000
59/59 [==============================] - 11s 179ms/step - loss: 0.4994 - val_loss: 0.4447
Epoch 7/1000
59/59 [==============================] - 11s 178ms/step - loss: 0.4490 - val_loss: 0.4161
Epoch 8/1000
59/59 [==============================] - 11s 178ms/step - loss: 0.4368 - val_loss: 0.4472
Epoch 9/1000
59/59 [==============================] - 11s 179ms/step - loss: 0.4268 - val_loss: 0.4130
Epoch 10/1000
59/59 [==============================] - 11s 178ms/step - l

...

wandb: wandb version 0.12.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-10-03 11:07:50.438157: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-03 11:07:50.438563: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



Epoch 1/1000
59/59 [==============================] - 66s 571ms/step - loss: 4.3079 - val_loss: 1.4177
Epoch 2/1000
59/59 [==============================] - 11s 179ms/step - loss: 1.2648 - val_loss: 0.7762
Epoch 3/1000
59/59 [==============================] - 11s 179ms/step - loss: 0.7825 - val_loss: 0.6396
Epoch 4/1000
59/59 [==============================] - 11s 179ms/step - loss: 0.6387 - val_loss: 0.6718
Epoch 5/1000
59/59 [==============================] - 11s 178ms/step - loss: 0.5748 - val_loss: 0.4758
Epoch 6/1000
59/59 [==============================] - 11s 178ms/step - loss: 0.4890 - val_loss: 0.4925
Epoch 7/1000
59/59 [==============================] - 11s 179ms/step - loss: 0.4821 - val_loss: 0.4226
Epoch 8/1000
59/59 [==============================] - 11s 179ms/step - loss: 0.4592 - val_loss: 0.4576
Epoch 9/1000
59/59 [==============================] - 11s 178ms/step - loss: 0.4219 - val_loss: 0.3996
Epoch 10/1000
59/59 [==============================] - 11s 179ms/step - l

best_epoch,128
best_val_loss,0.17452
epoch,999
loss,0.08541
lr,0.0
val_loss,0.17534


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,███▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [12]:
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

In [13]:
submission["pressure"] = sum(test_preds)/len(test_preds)
submission.to_csv('submission23.csv', index=False)